In [1]:
from openai import OpenAI

"""
TextGeneration Class Summary:

The TextGeneration class facilitates text generation using the OpenAI GPT-3.5 model. 

Attributes:
- client: An instance of the OpenAI API used for text generation.

Methods:
- __init__(self): Initializes the TextGeneration class with a specified topic for text generation.
- generate_json(self, user_prompt, system_prompt=None): Generates a JSON response based on user prompts and an optional system prompt
- generate_text(self, user_prompt, system_prompt=None): Generates a text-based response based on user prompts and an optional system prompt

Usage:
1. Instantiate TextGeneration 
2. Utilize the generate_json method to generate JSON responses based on user and system prompts.
3. Utilize the generate_text method to generate text-based responses based on user and system prompts.

"""

class GPTWrapper:
    def __init__(self):
        self.client = OpenAI()

    def generate_html(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows a lot about {topic} and only responds with HTML with only the HTML code. Nicely format with different header sizes and tables when necessary"

        response = self.client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content

    def generate_json(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows a lot about {topic} and only responds with JSON"

        return self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )

    def generate_text(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows a lot about {topic}"

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content

    def generate_dockerfile(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows how to make a Dockerfile that will achieve the following output: {topic}. I would like you to respond with only the text for a Dockerfile nothing else. No extra code or files allowed, but you can write inline code in the Dockerfile. Ensure that the dockerfile is runnable"

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content.strip("`").strip("Dockerfile").strip('dockerfile')

    def refine_dockerfile(self, user_prompt, dockerfile_string, build_logs, run_logs ,errors , topic="everything"):

        system_prompt = f"You are a helpful assistant that knows how to improve this existing dockerfile ({dockerfile_string}) so that it will achieve the following output: {topic}. No extra code or files allowed, but you can write inline code in the Dockerfile. I would like you to respond with only the text for the improved Dockerfile nothing else. Ensure that the dockerfile is runnable. Here are the build logs from when I built the image: {build_logs} and the runtime logs: {run_logs}. If any, here are the errors that occurred: {errors}. Please take action on any feedback and fix any problems "

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content.strip("`").strip("Dockerfile").strip('dockerfile')



## ProcessRunner Class
Wrapper around docker to handle the contained process factory

In [189]:
import shutil
import uuid
import os
from docker import DockerClient, errors

class DockerRunner:
    def __init__(self):
        self.docker_client = DockerClient(base_url='unix://var/run/docker.sock')

    def run_from_image_id(self, image_id, run_duration=5):
        container = self.docker_client.containers.run(image_id, detach=True)
        container.stop()
        # Capture runtime logs
        logs = container.logs(timestamps=True).decode('utf-8')
        # Wait for the specified run duration
        container.wait(timeout=run_duration)
        container.remove()
        response = {}
        response["image"] = image
        response["run_logs"] = logs
        return response

    def run_dockerfile(self, dockerfile_path, run_duration=5):
        # Generate a random directory to store the Dockerfile
        dir_path = f"temp/{str(uuid.uuid4())}"
        os.makedirs(dir_path, exist_ok=True)
        temp_dockerfile_path = os.path.join(dir_path, "Dockerfile")
        
        # Prepare the response dictionary
        response = {"dockerfile": dockerfile_path}
        with open(dockerfile_path, 'r') as file: data = file.read() 
        response["dockerfile_content"] = data
        
        try:
            # Copy Dockerfile to the temporary directory
            shutil.copy(dockerfile_path, temp_dockerfile_path)
            
            # Build the Docker image
            # image, build_logs = self.docker_client.images.build(path=dir_path, rm=True, forcerm=True)
            image, build_logs = self.docker_client.images.build(path=dir_path)
            
            # Run the container
            container = self.docker_client.containers.run(image.id, detach=True)
            
            # Wait for the specified run duration
            container.wait(timeout=run_duration)
            
            # Capture runtime logs
            logs = container.logs(timestamps=True).decode('utf-8')

            response["build_logs"] = [log.get('stream', '') for log in build_logs if 'stream' in log]
            response["image"] = image
            response["run_logs"] = logs
            
            # Stop and remove the container
            container.stop()
            container.remove()
            
        except (errors.DockerException, errors.APIError) as e:
            print(f"Error: {e}")
            response["error"] = str(e)
        
        finally:
            # Cleanup: remove the directory after building
            shutil.rmtree(dir_path)
        
        return response


In [190]:

dockerfile_path = './Dockerfile'
docker_runner = DockerRunner()
result = docker_runner.run_dockerfile(dockerfile_path)

In [191]:

docker_runner.run_from_image_id(result["image"].id)

{'image': <Image: ''>,
 'run_logs': '2024-04-04T01:49:30.755416790Z Thu Apr  4 03:49:30 CEST 2024\n'}

## ProcessBuilder Class

Builds the process including documentation and any prerequisite information for the artifact (dockerized process) to be consumed by the user

In [208]:
import io
from pprint import pp

class ProcessBuilder:
    def __init__(self, llm, task : str, epochs=1 ):
        self.llm = llm
        self.task = task
        self.epochs = epochs
        self.dockerfile_path = './Dockerfile'
        self.build_initial()

    def build_initial(self):
        dockerfile_lines = [self.llm.generate_dockerfile(self.task)]
        with open(self.dockerfile_path, 'w') as file:
            file.writelines(dockerfile_lines)

    def run_dockerfile(self):
        docker_runner = DockerRunner()
        result = docker_runner.run_dockerfile(self.dockerfile_path)
        return result

    def iterate(self, new_result, user_feedback):
        refined_dockerfile_lines = self.llm.refine_dockerfile(
            user_feedback, 
            dockerfile_string=new_result.get('dockerfile_content'),
            build_logs=new_result.get('build_logs not available', None),
            run_logs=new_result.get('run_logs', "run logs not available"),
            errors=new_result.get("error", "no errors"),
            topic=self.task
        )
        with open(self.dockerfile_path, 'w') as file:
            file.write(refined_dockerfile_lines)
        return self.run_dockerfile()

    def documented(self, result):
        result['documentation'] = self.llm.generate_text(f"I am writing an api endpoint for the given task {self.task}, write a 10 word description for the enpoint including the request type, description and any inputs. Respond with only the 10 words")
        result['endpoint_name'] = self.llm.generate_text(f"I am writing an api endpoint for the given task {self.task}, respond with the name of the endpoint only in one word ")
        result['task'] = self.task
        result['epochs'] = self.epochs
        result.pop("build_logs")
        result.pop("run_logs")
        # result['run_logs'] = io.StringIO(result["run_logs"])
        # result['build_logs'] = io.StringIO(result["build_logs"])
        return result

    def build_process(self) -> dict: 
        result = {'dockerfile_content': None}  # Initialize result dictionary
        for _ in range(self.epochs):
            feedback = self.llm.generate_text(f"in 20 words describe what went wrong")
            result = self.iterate(new_result=result, user_feedback=feedback)
            if 'error' not in result or not result['error']:  # Assuming 'error' key presence indicates issues
                break  # Exit loop if no errors in the latest iteration
        return self.documented(result)

In [210]:
process = ProcessBuilder(llm = GPTWrapper(), task = "get the time in italy",  epochs= 1).build_process()
pp(process)

{'dockerfile': './Dockerfile',
 'dockerfile_content': 'FROM alpine:3.14\n'
                       '\n'
                       '# Set the timezone to Italy\n'
                       'RUN apk add --no-cache tzdata\n'
                       'ENV TZ=Europe/Rome\n'
                       '\n'
                       'CMD ["date"]',
 'image': <Image: ''>,
 'documentation': 'API endpoint to get Italy time, using GET request method.',
 'endpoint_name': '"italyTime"',
 'task': 'get the time in italy',
 'epochs': 1}


In [211]:
## Batch Process builder

In [240]:
fns = [
    "generate a random number from 0 to 100",
    "get time in italy",
    "generate the first 16 numbers of pi",
    "print the hex for the color blue",
    "generate the first 16 numbers of pi",
]
processes = [
    ProcessBuilder(llm=GPTWrapper(), task=fn, epochs=3).build_process() for fn in fns
]

## AsyncFunctionGenerator
generate a list of async functions that run the built images

In [244]:
from typing import List, Callable, Coroutine
import asyncio

class AsyncFunctionGenerator:
    def __init__(self, function_names: List[str], image_list: List[str]):
        self.functions: List[Callable[..., Coroutine]] = []
        self._generate_functions(function_names, image_list=image_list)

    def _generate_function(self, name: str, image: str) -> Callable[..., Coroutine]:
        # Prepare the function body with the image variable correctly interpolated
        async_template = f"""
async def {name}():
    print("hello from '{image}'")
    import subprocess; subprocess.run(["docker", "run", "{image}"])
"""
        local_namespace = {}
        # Pass the globals and local_namespace as the environment for the exec
        exec(async_template, globals(), local_namespace)
        # The created function is now stored in the local_namespace with the key as its name
        return local_namespace[name]

    def _generate_functions(self, function_names: List[str], image_list: List[str]):
        for name, image in zip(function_names, image_list):
            # Generate a function for each name, passing along the corresponding image string
            self.functions.append(self._generate_function(name, image))



In [253]:
# [process for process in processes]
files= [process["dockerfile_content"].strip("\"") for process in processes]
print("########## DOCKERFILES ############")
for file in files:
  print("------------------")
  pp(file)
  print("------------------")

print("########## FUNCTION:imageid ############")
pp({ process["endpoint_name"].strip("\"") : process["image"] .id for process in processes} )

images = [process["image"].id for process in processes]
names = [process["endpoint_name"].strip("\"") for process in processes]


########## DOCKERFILES ############
------------------
'FROM alpine:latest\n\nCMD ["sh", "-c", "echo $(( $RANDOM % 101 ))"]'
------------------
------------------
('FROM alpine\n'
 '\n'
 'RUN apk add --no-cache tzdata\n'
 'ENV TZ=Europe/Rome\n'
 '\n'
 'CMD ["date"]')
------------------
------------------
('FROM alpine:latest\n'
 '\n'
 'RUN apk add --no-cache bc\n'
 '\n'
 'CMD echo "scale=16; a(1)*4" | bc -')
------------------
------------------
"FROM alpine\nRUN echo -n '#0000FF'"
------------------
------------------
'FROM alpine\n\nRUN apk add --no-cache bc\n\nCMD echo "scale=16; 4*a(1)" | bc -'
------------------
########## FUNCTION:imageid ############
{'randomNumberEndpoint': 'sha256:813c9adc9b04112e86678071ab23941ddb3f4cafcf81e8e10adf6e344a3763d4',
 'timeInItaly': 'sha256:e96b2129db1844ccdb2a573e45c2023de633f490ba9f147d1f30ccbb7ede2d72',
 'piEndpoint': 'sha256:0609121db6f4ba7fae86bc3fb482fbef7cb126e6800a3f8d26ee022f123591c7',
 'colorHex': 'sha256:2987ab2a8c4ac0d8f180632c4fe4c82f

In [264]:
# Example usage
generator = AsyncFunctionGenerator(names, images)

async def main():
    for func in generator.functions:
        await func()  # This will execute each function and print the message

# Corrected the execution with the fixed class definition
await main()
# generator.functions[1].__name__

{'/randomNumberEndpoint': <function __main__.randomNumberEndpoint()>,
 '/timeInItaly': <function __main__.timeInItaly()>,
 '/piEndpoint': <function __main__.piEndpoint()>,
 '/colorHex': <function __main__.colorHex()>}

# HoudiniApi
The final product: an API built purely from prompts

In [258]:
import asyncio
from fastapi import FastAPI, APIRouter

# Define your endpoint functions outside the class
async def root():
    return {"message": "Hello World from the function"}

async def run_subprocess():
    # Example: Running 'echo' command. Replace with your actual command
    process = await asyncio.create_subprocess_shell(
        'echo "Hello from subprocess"',
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE)

    stdout, stderr = await process.communicate()

    if stderr:
        return {"error": stderr.decode()}
    
    return {"message": stdout.decode().strip()}

class APIEndpoints:
    def __init__(self, endpoint_functions):
        self.router = APIRouter()
        self.endpoint_functions = endpoint_functions
        self._register_routes()

    def _register_routes(self):
        # Iterate through the list of functions passed to the constructor
        for path, func in self.endpoint_functions.items():
            self.router.add_api_route(path, self._wrap_async(func), methods=["GET"])

    def _wrap_async(self, func):
        # This wrapper converts the standalone async function to a method that can be called on the class instance.
        async def wrapped():
            return await func()
        return wrapped

# Initialize FastAPI app
app = FastAPI()


In [269]:

# Create an instance of your APIEndpoints class, passing a dictionary of endpoint functions
api_endpoints = APIEndpoints({ "/"+fn.__name__:fn  for fn in generator.functions})
# Include the routes from the APIEndpoints instance into your FastAPI app
api_endpoints.endpoint_functions

{'/randomNumberEndpoint': <function __main__.randomNumberEndpoint()>,
 '/timeInItaly': <function __main__.timeInItaly()>,
 '/piEndpoint': <function __main__.piEndpoint()>,
 '/colorHex': <function __main__.colorHex()>}

In [270]:

app.include_router(api_endpoints.router)